In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session~
#a lward ifires djilali d ahwawi ihemmey adihewes

/kaggle/input/are-sd-2025-classification-diagnostique-d-arbres/benchmark.csv
/kaggle/input/are-sd-2025-classification-diagnostique-d-arbres/prev.csv
/kaggle/input/are-sd-2025-classification-diagnostique-d-arbres/train.csv


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
data = pd.read_csv('/kaggle/input/are-sd-2025-classification-diagnostique-d-arbres/train.csv', index_col='ID_ARBRE')
prev_data = pd.read_csv('/kaggle/input/are-sd-2025-classification-diagnostique-d-arbres/prev.csv', index_col='ID_ARBRE')
prev_data.head(3)

,quartier,site,genre_arbre,situation,type_sol,surf_permeable,classe_age,hauteur,classe_hauteur,diametre,...,fissure_houppier,ecorce_incluse_houppier,bois_mort_houppier,plaie_houppier,observation_houppier,esperance_maintien,contrainte,Long,Lat,Usage
ID_ARBRE,,,,,,,,,,,,,,,,,,,,,
559,Quartier 2 - Alsace - Pereire,Avenue du Maréchal Foch,Tilia,Alignement,MA,4.0,A,800,H2,44.563384,...,HPF,Non,HPBM,HPLNC,Branches cassées,2.0,Non,2.084,48.904,Private
560,Quartier 4 - Rotondes - St Léger,Avenue Saint-Fiacre,Carpinus,Alignement,Gr,1.0,JA,800,H2,38.197186,...,HPF,Non,HPBM,HPLS,0,1.0,Non,2.072,48.895,Private
561,Quartier 2 - Alsace - Pereire,Avenue du Maréchal Foch,Tilia,Alignement,MA,4.0,A,800,H2,63.661977,...,HPF,Non,HPBM,HPLS,0,2.0,Non,2.083,48.904,Private


***Code du modèle BAIES NAIF***

In [4]:
def pxy(data, var):
    """Calcul les probabilités jointes P(XetY)"""
    return  np.array( [ [ np.mean( (data[var] == X) * (data['classification_diagnostic'] == Y) ) for X in np.unique(data[var]) ] for Y in ['C1', 'C2', 'C3', 'C4', 'C5'] ] )

def pxsy (data, var):
    """Calcul les p"""
    return  np.array( [ [  np.sum((data[var] == X) & (data['classification_diagnostic'] == Y)) / np.sum(data['classification_diagnostic'] == Y) for X in np.unique(data[var]) ] for Y in ['C1', 'C2', 'C3', 'C4', 'C5'] ] )


In [5]:
"""np.unique(prev_data['classe_age']), np.unique(data['classe_age']), data.columns
for i in data.columns:
    if i != 'classification_diagnostic' and i != 'esperance_maintien':
        #print(len(np.unique(prev_data[i])) == len(np.unique(data[i])))
        if len(np.unique(prev_data[i])) != len(np.unique(data[i])):
            print(i)"""
        

"np.unique(prev_data['classe_age']), np.unique(data['classe_age']), data.columns\nfor i in data.columns:\n    if i != 'classification_diagnostic' and i != 'esperance_maintien':\n        #print(len(np.unique(prev_data[i])) == len(np.unique(data[i])))\n        if len(np.unique(prev_data[i])) != len(np.unique(data[i])):\n            print(i)"

In [6]:
def pxsy_lissage_laplace (classes_x, train_data, var):
    k = len(classes_x)
    return  np.array( [ [  (1+np.sum((train_data[var] == X) & (train_data['classification_diagnostic'] == Y))) / (np.sum(train_data['classification_diagnostic'] == Y)+k) for X in classes_x] for Y in ['C1', 'C2', 'C3', 'C4', 'C5'] ] ) 

In [7]:
def vraissemblance(train_data, prev_data, pxsy_f, vvars):
    classes = ['C1', 'C2', 'C3', 'C4', 'C5']
    valeurs_possibles = {var: np.unique(prev_data[var]) for var in vvars}
    #calcul des probabilitées p(Y)
    py = np.array([np.mean(train_data['classification_diagnostic'] == Y) for Y in classes])

    #calcul des p(X|Y)
    pxsy_t = []
    for var in vvars:
        pxsy_t.append(pxsy_f(valeurs_possibles[var], train_data, var))
    
    #calcul des probabilitées p(XetY)
    #la case (i, j) de pxy correspond à p(X|Y='Cj') pour le i-ème arbre
    pxy = np.zeros((len(prev_data), 5)) 
    for i in range(len(prev_data)):
        for c in range(5):
            ptemp = 1
            for pxsy, var in zip(pxsy_t, vvars):
                valeur = prev_data.iloc[i][var]
                index_val = np.where(valeurs_possibles[var] == valeur)[0][0]
                ptemp = ptemp * pxsy[c, index_val]
            pxy[i, c] = ptemp * py[c]
    return pxy

In [8]:
def predict_baies_naif (prev_data, train_data, vvars, pxsy_f):
    """Predit la calsse des arbres de prev_data avec le modèle BAIES NAIF
    prev_data : les données des arbres à prédire 
    train_data : les données des arbres qui servent comme données d'entrainement
    vvars : liste des variables utilisées pour prédire 
    pxsy_f : fonction permettant de calculer les probabilités conditionnelles des variables sacant la classe"""
    
    pxy = vraissemblance(train_data, prev_data, pxsy_f, vvars)
    return ['C' + str( 1+np.argmax( pxy[i] ) ) for i in range( len(prev_data) )]
        

In [9]:
def validation_croisée_baies_naif(data: pd.DataFrame, predi_fonc, v_vars: np.array , pxsy_f, n_iter: int):
    """Renvoie le taux de réussite de la prediction du niveau de blessure des arbres par la fonction predi_f 
    testé sur data_test.
    """
    long = len(data) // 5
    res = []
    for i in range(n_iter):
        v_rd = np.random.permutation(data.index)
        data_test = data[data.index.isin(v_rd[:long])]
        data_train = data[data.index.isin(v_rd[long:])]
        prediction = predi_fonc(data_test, data_train, v_vars, pxsy_f)
        realite = data_test["classification_diagnostic"].to_numpy()
        res.append(np.mean(prediction == realite) * 100)
    return np.mean(res)


In [10]:
"""from tqdm import tqdm
vars_tout = [var for var in data.columns if var!='classification_diagnostic' and var!='esperance_maintien']

dico1 = {"vars" : [], "score" : []}
for i in tqdm(range(200)):
    rd = np.random.permutation (len(vars_tout))
    vars_ch = [vars_tout[i] for i in rd[:5]]
    sc = validation_croisée_baies_naif(data, predict_baies_naif, vars_ch, pxsy_lissage_laplace, 20)
    dico1["vars"].append(vars_ch) 
    dico1["score"].append(sc)"""

'from tqdm import tqdm\nvars_tout = [var for var in data.columns if var!=\'classification_diagnostic\' and var!=\'esperance_maintien\']\n\ndico1 = {"vars" : [], "score" : []}\nfor i in tqdm(range(200)):\n    rd = np.random.permutation (len(vars_tout))\n    vars_ch = [vars_tout[i] for i in rd[:5]]\n    sc = validation_croisée_baies_naif(data, predict_baies_naif, vars_ch, pxsy_lissage_laplace, 20)\n    dico1["vars"].append(vars_ch) \n    dico1["score"].append(sc)'

****FICHIER POUR LA SOUMISSION****

In [11]:
predict = predict_baies_naif (prev_data, data,  ['genre_arbre', 'classe_hauteur', 'champignon_collet', 'plaie_tronc', 'Long'], pxsy_lissage_laplace)
index = np.array([i for i in prev_data.index])
v_res = np.vstack((index, predict)).T
df = pd.DataFrame(v_res, columns=[['ID_ARBRE', 'classification_diagnostic']])
df.to_csv("vpredi.csv", index = False)
df

,ID_ARBRE,classification_diagnostic
0,559,C2
1,560,C2
2,561,C2
3,562,C1
4,563,C4
...,...,...
132,694,C2
133,695,C3
134,696,C2
135,697,C1
